In [ ]:
import numpy as np
import pandas as pd

from IPython.display import Markdown, display

from tqdm.notebook import tqdm

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [ ]:
RANDOM_SEED = 42
VAL_SIZE   = 0.20

In [ ]:
def printmd(string):
    '''Продвинутый вывод строк'''
    display(Markdown(string))



def mape(y_true, y_pred):
    '''Ключевая метрика'''
    return np.mean(np.abs((y_pred-y_true)/y_true))


def search_bad_features(df, goal, model, metric_value, col_lst, r=2, cv=4):
    '''Ищет признаки, ухудшающие модель

    Удаляет признак, обучает модель и сверяет метрики.
    Если показатели улучшились, удаляет признак из изначальной выборки
    и снова запускает цикл, пока метрики не перестанут улучшаться.
    Параметр r отвечает за точность метрик.
    '''
    
    min_mape = round(metric_value, r)
    drop_lst = []
    cnt = 0
    while True:
        temp_lst = []
        for i, col in enumerate (tqdm(col_lst, leave=False, desc='Cols')):
            if len(temp_lst) != 0:
                continue

            X_func = df.drop([goal]+[col]+drop_lst, axis=1).reset_index(drop=True)
            y_func = df[goal].values

            temp_min_mape = round(kfold_validation(model, X_func, y_func, cv=cv), r)

            if temp_min_mape < min_mape:
                min_mape = temp_min_mape
                temp_lst.append(col)
                printmd(f"*Столбец для удаления:* ***{col}.***")
                printmd(f"*Значение метрики* ***MAPE - {min_mape}***")

        drop_lst += temp_lst
        for drop in drop_lst:
            if drop in col_lst:
                col_lst.remove(drop)
        if cnt != len(drop_lst):
            cnt = len(drop_lst)
        else:
            break

    if len(drop_lst) == 0:
        printmd("**Нет признаков для удаления**")
    else:
        drop_cols = ', '.join(drop_lst)
        printmd(f"Признаки для удаления: **{drop_cols}**")

    return drop_lst



def kfold_validation(model, X, y, cv=4): 
    '''Кросс-валидация алгоритма'''
    skf = KFold(n_splits=cv, shuffle=True, random_state=RANDOM_SEED)
    mape_values = []

    for train_index, test_index in tqdm(skf.split(X, y), leave=False, desc='kv'):
        X_train = X.filter(items=train_index, axis=0)
        X_test = X.filter(items=test_index, axis=0)
        y_train = y[train_index]
        y_test = y[test_index]

        # For training, fit() is used
        model.fit(X_train, np.log(y_train))

        # For MAPE metric (or any other), we need the predictions of the model
        y_pred = np.exp(model.predict(X_test))

        mape_value = mape(y_test, y_pred)
        mape_values.append(mape_value)

    return np.mean(mape_values)

In [ ]:
'''Предобработка датасета'''
data = pd.read_csv('my_data.csv')
data['year'] = data['data'].apply(lambda x: 1 if x == 'parsed' else 0)
data = data[data['data'] != 'test'].drop(['data']+['parsing_date_to_now'], axis=1)

In [ ]:
X = data.drop(['price'], axis=1).reset_index(drop=True)
y = data['price'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
'''Первичное обучение модели для получения начальной метрики'''
etr_log = ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1)
MAPE = kfold_validation(etr_log, X, y, cv=10)
printmd(f"*Значение метрики* ***MAPE - {(MAPE)*100:0.2f}%***")

In [ ]:
'''
Формирование списка признаков для для дальнейшей проверки.
Сортирует от наименее важного к наиболее.
'''
feat_importances = pd.Series(etr_log.feature_importances_, index=X.columns)
feat_cnt = len(pd.Series(etr_log.feature_importances_, index=X.columns))
feat_lst = list(feat_importances.nsmallest(feat_cnt).index)

In [ ]:
'''Получение списка признаков на удаление'''
drop_lst = search_bad_features(data, 'price', etr_log, MAPE, feat_lst, r=5, cv=10)
print(drop_lst)